# Practical 1: Introduction to Distributed Systems

**Course**: BMCS3003 Distributed Systems and Parallel Computing  
**Difficulty**: ⭐⭐  
**Estimated Time**: 90 minutes  
**Prerequisites**: Basic C++ programming, socket programming concepts

## Learning Objectives
By the end of this notebook, you will be able to:
1. Understand Cristian's clock synchronization algorithm
2. Implement client-server socket communication in C++
3. Calculate network latency and synchronization error
4. Compare latency between localhost and network communication
5. Explore Berkeley's algorithm for distributed clock synchronization

## 1. Introduction to Parallel Processing Technologies

Throughout this course, we will explore various parallel processing mechanisms in C++:

- **Threading**: C++ standard library threads
- **PThread**: POSIX threads for cross-platform threading
- **OpenMP**: Compiler directives for shared-memory parallelism
- **MPI**: Message Passing Interface for distributed computing
- **CUDA**: GPU parallel computing platform

### Why Start with Clock Synchronization?

Before diving into parallel processing, we need to understand how distributed systems communicate and synchronize. Clock synchronization is fundamental because:
- Different computers have different clock times
- Network delays affect communication
- Coordinating parallel tasks requires synchronized time

## 2. Cristian's Clock Synchronization Algorithm

### What is Cristian's Algorithm?

Cristian's Algorithm is a method for synchronizing time between a **time server** and **client processes** in a distributed system.

### How It Works

```
        CLOCK SERVER
            T_SERVER
               •
              /|\
    REQUEST  / | \ RESPONSE
            /  |  \
           /   |   \
          •————————•
         T0    RTD   T1
      PROCESS/CLIENT
```

### Algorithm Steps

1. **Client sends request** at time `T0`
2. **Server receives request** and responds with server time `T_SERVER`
3. **Client receives response** at time `T1`
4. **Client calculates synchronized time**:

```
T_CLIENT = T_SERVER + (T1 - T0) / 2
```

### Why This Formula Works

- `(T1 - T0)` = Round Trip Time (RTT)
- `(T1 - T0) / 2` = Estimated one-way latency (assuming symmetric network)
- **Synchronization error** ≤ `(T1 - T0) / 2` seconds

### When to Use Cristian's Algorithm

✅ **Best for**:
- Low-latency networks (short RTT)
- Single time server architecture
- Applications where millisecond accuracy is sufficient

❌ **Not suitable for**:
- High-latency or unstable networks
- Redundancy-critical systems (single point of failure)
- Asymmetric network paths (different upload/download speeds)

## 3. Understanding Socket Communication

### What is a Socket?

A **socket** is an endpoint for sending or receiving data across a network. Think of it as a "phone line" between programs.

### Server-Client Communication Flow

#### Server Side:
1. **Create socket** → `socket()`
2. **Bind to IP address and port** → `bind()`
3. **Listen for connections** → `listen()`
4. **Accept client connection** → `accept()`
5. **Send/Receive data** → `send()` / `recv()`
6. **Close connection** → `closesocket()`

#### Client Side:
1. **Create socket** → `socket()`
2. **Connect to server** → `connect()`
3. **Send/Receive data** → `send()` / `recv()`
4. **Close connection** → `closesocket()`

### Important Concepts

- **IP Address**: Identifies the machine (e.g., `127.0.0.1` for localhost)
- **Port Number**: Identifies the application (e.g., `9999`)
- **Localhost (`127.0.0.1`)**: Refers to your own computer
- **Intranet IP** (e.g., `192.168.1.100`): Internal network address

## 4. Question 1: Implementing Cristian's Algorithm

### Task Overview

Implement a clock server and client using Cristian's algorithm to:
1. Establish socket connection between server and client
2. Exchange time information
3. Calculate synchronization
4. Measure network latency

### Part A: Server Code Explanation

```cpp
// Key Configuration
int port = 9999;                    // Port number for communication
const char* ipadd = "127.0.0.1";    // Localhost IP address

// Server waits in infinite loop
while (true) {
    // Accept client connection
    int client_sockfd = accept(server_sockfd, ...);
    
    // Get current server time
    auto now = std::chrono::system_clock::now();
    std::time_t time = std::chrono::system_clock::to_time_t(now);
    
    // Send time to client
    send(client_sockfd, time_ch, time_str.size(), 0);
}
```

### Part B: Client Code Explanation

```cpp
// Record request time (T0)
auto request_time_point = std::chrono::system_clock::now();

// Receive server time
recv(server_sockfd, ch, 24, 0);

// Record response time (T1)
auto response_time_point = std::chrono::system_clock::now();

// Calculate latency = T1 - T0
auto latency_time = response_time_epoch - request_time_epoch;

// Synchronized time = T_SERVER + latency/2
std::time_t client_time = actual_time; // + (latency_time / 2);
```

## 5. Running the Code

### Setup Instructions

#### Step 1: Create Visual Studio Projects
1. Create two C++ projects: `P1Q1server` and `P1Q1client`
2. Copy the server code into `P1Q1server.cpp`
3. Copy the client code into `P1Q1client.cpp`

#### Step 2: Configure for Windows Sockets
Both projects need:
```cpp
#pragma comment(lib, "ws2_32.lib")  // Link Windows Socket library
```

#### Step 3: Test with Localhost
```cpp
// Server and Client both use:
const char* ipadd = "127.0.0.1";  // Localhost
int port = 9999;
```

1. **Run server first** (wait for "Server is waiting...")
2. **Run client** (will connect and receive time)

#### Step 4: Test Across Network
```cpp
// Server uses:
const char* ipadd = "0.0.0.0";  // Listen on all interfaces

// Client uses actual server IP:
const char* ipadd = "192.168.1.100";  // Replace with server's IP
```

### Expected Output

**Localhost (Tight Architecture)**:
```
Process Delay latency: 1700 microseconds
Actual clock time at client side: Tue Jul 4 11:12:02 2023
Synchronized process client time: Tue Jul 4 11:12:02 2023
Synchronization error: 0 microseconds
```

**Network (Loose Architecture)**:
```
Process Delay latency: 6509 microseconds
Actual clock time at client side: Tue Jul 4 11:22:06 2023
Synchronized process client time: Tue Jul 4 11:22:06 2023
Synchronization error: 0 microseconds
```

## 6. Code Issues to Fix

### Problem 1: Time Conversion
```cpp
// PROBLEM: Cannot convert char* to time_t correctly
const char* time_ch = str.c_str();
std::time_t server_time = std::time_t(time_ch);  // ❌ WRONG!
```

**Fix**: Parse the time string properly or use binary time format
```cpp
// SOLUTION: Send time as binary data instead of string
std::time_t server_time = std::chrono::system_clock::to_time_t(now);
send(client_sockfd, (char*)&server_time, sizeof(server_time), 0);
```

### Problem 2: Subsecond Precision
```cpp
// PROBLEM: time_t has no decimal point for subsecond precision
std::time_t client_time = actual_time; // + (latency_time / 2);
```

**Fix**: Use `std::chrono` for high-resolution timing
```cpp
// SOLUTION: Work with time_point throughout
auto client_time_point = server_time_point + (latency_time / 2);
```

## 7. Analysis and Discussion

### Latency Comparison

| Architecture | Latency | Why? |
|--------------|---------|------|
| **Localhost** (127.0.0.1) | ~1.7 ms | No physical network, just kernel routing |
| **Intranet** (192.168.x.x) | ~6.5 ms | Physical network cables, switches, NICs |
| **Internet** | 50-500 ms | Multiple routers, long distances, congestion |

### Key Observations

1. **Network adds significant latency**: Even local network is 4x slower than localhost
2. **Latency affects synchronization accuracy**: Higher latency = higher potential error
3. **Asymmetric networks**: Upload/download speeds may differ, violating algorithm assumption

### Real-World Applications

- **Network Time Protocol (NTP)**: Uses similar principles but with multiple servers
- **Distributed databases**: Need synchronized clocks for transaction ordering
- **Video conferencing**: Synchronize audio/video streams across users
- **Financial trading**: Timestamp transactions accurately

## 8. Question 2: Berkeley's Algorithm

### How Berkeley's Algorithm Differs

**Cristian's Algorithm**: Server provides time, clients adjust to match server

**Berkeley's Algorithm**: Coordinator collects times from all nodes and computes average

### Algorithm Steps

1. **Coordinator polls all nodes** for their current time
2. **Calculate differences** between each node and coordinator
3. **Compute average time** (excluding outliers)
4. **Send adjustments** to each node (not absolute time)

### Example

```
Node A: 10:00:00
Node B: 10:00:05  (Coordinator)
Node C: 10:00:10

Average = (0 + 5 + 10) / 3 = 5 seconds

Adjustments:
A: +5 seconds
B: 0 seconds
C: -5 seconds
```

### Advantages over Cristian's

- No single point of failure (coordinator can be elected)
- More accurate with multiple nodes
- Handles faulty clocks better (can exclude outliers)

### Implementation Task

Implement Berkeley's algorithm in C++ with:
- 1 coordinator process
- 3-5 client processes
- Socket communication for time exchange
- Average calculation with outlier detection

**Note**: No solution is provided - this is your challenge!

## 9. Summary

### Key Concepts Learned

1. **Clock Synchronization**: Essential for distributed systems coordination
2. **Cristian's Algorithm**: Simple client-server time synchronization
3. **Socket Programming**: Client-server communication using TCP sockets
4. **Network Latency**: Significantly impacts synchronization accuracy
5. **Berkeley's Algorithm**: More robust alternative using consensus

### Formula to Remember

```
T_CLIENT = T_SERVER + (T1 - T0) / 2

Synchronization Error ≤ (T1 - T0) / 2
```

### Next Steps

In **Practical 2**, we'll explore:
- C++ standard threads
- POSIX threads (pthreads)
- Thread synchronization
- UDP communication (faster but unreliable)

## 10. Additional Resources

### References
- [Cristian's Algorithm - GeeksforGeeks](https://www.geeksforgeeks.org/cristians-algorithm/)
- [Berkeley's Algorithm - GeeksforGeeks](https://www.geeksforgeeks.org/berkeleys-algorithm/)
- [Winsock2 Documentation](https://docs.microsoft.com/en-us/windows/win32/winsock/)

### Practice Exercises

1. **Modify latency calculation**: Account for clock drift by adding a drift factor
2. **Multiple requests**: Client sends multiple requests and averages results
3. **Timeout handling**: Add timeouts for unresponsive servers
4. **UDP version**: Implement using UDP instead of TCP (faster but less reliable)
5. **Visualization**: Plot latency over time to observe network behavior